In [ ]:
!pip install ussa1976

In [ ]:
QUICK_TEST = True

#
# 0.  Libraries
#
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import minimize, Bounds, NonlinearConstraint
import ussa1976 as usa

#
# 1.  Decide which numbers are the "knobs"
#


#
kN = 1_000
g0 = 9.81                                  # m s-2
P0 = 101325.0                               # Pa
R_E  = 6_371_000.0          # m, mean Earth radius
μ_E  = 3.986_004_418e14

n_booster = 2 #
n_engines_booster = 9
Thrust_booster_SL = 845 # sea level
Thrust_booster_Vac = 914

BOOSTER = dict(
    T_SL       = n_booster * n_engines_booster * Thrust_booster_SL * kN, # N   (18 SRMs @ 845 kN each)
    T_Vac       = n_booster * n_engines_booster * Thrust_booster_Vac * kN, # N   (18 SRMs @ 845 kN each)
    Isp_SL     = 282, # s
    Isp_Vac     = 311, # s
    burn    = 150, # s
    m_prop  = 410_900 * n_booster, # kg  (two boosters)
    m_dry   = 22_200 * n_booster # kg
)

BOOSTER['A_e'] = (BOOSTER['T_Vac'] - BOOSTER['T_SL']) / P0

n_engines_core = 9
Thrust_core_SL = 845 # sea level
Thrust_core_Vac = 914

CORE = dict(
    T_SL       = n_engines_core * Thrust_core_SL * kN, # N 9 Merlin engines
    T_Vac       = n_engines_core * Thrust_core_Vac * kN, # N 9 Merlin engines
    Isp_SL     = 282, # s
    Isp_Vac     = 311, # s Second stage engine has larger expansion ratio
    burn    = 211, # s
    m_prop  = 410_900, # kg
    m_dry   = 22_200 # kg
)

CORE['A_e'] = (CORE['T_Vac'] - CORE['T_SL']) / P0

SECOND = dict(
    m_prop = 111_500,     # kg  (Merlin-Vac prop load; rough Falcon-9 upper)
    m_dry  =  4_000, # kg  (dry hardware)
    T_SL = 934 * kN,
    T_vac  = 981 * kN,    # not used yet, but nice to store
    Isp_vac= 348,         #  –
    burn_1 = 306,          # From first ignition to SECO 1
    burn_2   = 800,          #  From first ignition to SECO 2
    A_e = 4.9 # m2
)

SECOND['A_e'] = (SECOND['T_vac'] - SECOND['T_SL']) / P0

# Add its total mass to the payload the first stage is lifting

PAYLOAD_MAX = 1000          # kg
PAYLOAD_GUESS = 10
ALT_TARGET  = 1_000_000.0 # m

# Liftoff_mass = CORE['m_prop'] + CORE['m_dry'] + BOOSTER['m_prop'] + BOOSTER['m_dry'] + PAYLOAD

#T_delay = 0.2 # 20% delay between stages
T_IGN = 222 # from PDF for stage 2
T_IGN_1 = T_IGN                                # 222 s
SECO1   = T_IGN_1 + SECOND['burn_1']           # 528 s
T_IGN_2 = 750                                 # restart
SECO2   = T_IGN_1 + SECOND['burn_2']          # 1740 s



pitch_knots  = np.array([ 0,  20,  60, 180, 380, SECO1, SECO2])
pitch_values = np.radians([90, 75, 50, 35, 27, 20, 18])


#  throttle knot times, user defined
knots_booster  = np.array([0,  60, BOOSTER['burn'] - 10, BOOSTER['burn']])
knots_core     = np.array([0,  60, CORE['burn'] - 10, CORE['burn']])
knots_second   = np.array([0, 150, 306, 1432, 1518])


#  initial guesses (anything in [0.4,1])
guess_booster = np.linspace(1.0, 0.0, len(knots_booster))
guess_core = np.linspace(1.0, 0.0, len(knots_core))
guess_second = np.linspace(1.0, 0.0, len(knots_second))


guess_booster[-1] = 0.0
guess_core   [-1] = 0.0
guess_second[-1] = 0.0

# flatten them into a single parameter vector
theta0 = np.concatenate([guess_booster, guess_core, guess_second, [PAYLOAD_GUESS/PAYLOAD_MAX]])


# helper slices so we can pull them back out of θ later
sl_b = slice(0, len(knots_booster))
sl_c = slice(sl_b.stop, sl_b.stop + len(knots_core))
sl_s = slice(sl_c.stop, sl_c.stop + len(knots_second))
p_idx = -1


d_core = 3.66
h_core = 70
#A_core = surface_area(d_core,h_core)

d_booster = 3.66
h_booster = 41.2
#A_booster = surface_area(d_booster,h_booster)

h_core_2 = h_core - h_booster

#
# 2.  Helpers

Z     = np.arange(0.0, 400_001.0, 100.0) # 0-400 km, 100 m
atm   = usa.compute(z=Z, variables=["rho", "p"])  # get density & pressure

Z_ext  = np.append(atm.z.values,        [10_000_000.0])   # 10 000 km
p_ext  = np.append(atm.p.values,        [0.0])
rho_ext= np.append(atm.rho.values,      [0.0])

def pressure_ussa(h):
    return np.interp(h, Z_ext, p_ext)   # Pa

def density_ussa(h):
    return np.interp(h, Z_ext, rho_ext)

def gravity(h):
    return μ_E / (R_E + h)**2

def thrust_interp(stage, h):
    return stage['T_Vac'] - stage['A_e'] * pressure_ussa(h)

def isp_interp(stage, p):
    return stage['Isp_Vac'] - (stage['Isp_Vac'] - stage['Isp_SL']) * (p / P0)

# helpers that don’t change during optimisation

def second_throttle(t, throttle_second):

    # split once per call
    k1 = knots_second[knots_second <= SECOND['burn_1']]          # [0 … 306]
    t1 = throttle_second[:len(k1)]

    k2 = (knots_second[knots_second >= SECOND['burn_1']]
          - SECOND['burn_1'])                                    # [0 … 1212]
    t2 = throttle_second[-len(k2):]

    if (t < T_IGN) or (SECO1 <= t < T_IGN_2) or (t >= SECO2):
        return 0.0
    if t < SECO1:
        return np.interp(t - T_IGN,  k1, t1)          # first burn
    return np.interp(t - T_IGN_2, k2, t2)             # second burn


_sim_cache = {}


def simulate(theta):

    key = tuple(theta)   # hashable & robust to fp noise
    if key in _sim_cache:             #  instant return on 2nd call
        return _sim_cache[key]

    P = theta[p_idx] * PAYLOAD_MAX                # UNPACK
    assert P >= 0.0

    throttle_booster = theta[sl_b]
    throttle_core    = theta[sl_c]
    throttle_second  = theta[sl_s]

    # stage-specific throttle functions
    def f_booster(t):
        return np.interp(t, knots_booster, throttle_booster)

    def f_core(t):
        return np.interp(t, knots_core, throttle_core)

#    def f_second(t):
#        if t < T_IGN:                                   # engine off
#            return 0.0
#        return np.interp(t - T_IGN, knots_second, throttle_second)


    # derivatives with the new throttle functions
    def rhs(t, S):
        (x, z, vx, vz, m_b, m_c, m_s) = S
        v   = np.hypot(vx, vz) + 1e-9
        gam = np.interp(t, pitch_knots, pitch_values)

        p   = pressure_ussa(z)
        g   = gravity(z)

        # boosters
        boosters_on = (t <= BOOSTER['burn']) and (m_b > 0)
        if boosters_on:
            Tb_max  = thrust_interp(BOOSTER, z)
            Tb      = f_booster(t) * Tb_max
            mdot_b  = Tb / (isp_interp(BOOSTER, p) * g0)
            dmb_dt  = -mdot_b
        else:
            Tb, dmb_dt, m_b = 0.0, 0.0, 0.0

        # core
        core_on = (t <= CORE['burn']) and (m_c > 0)
        if core_on:
            Tc_max = thrust_interp(CORE, z)
            Tc     = f_core(t) * Tc_max
            mdot_c = Tc / (isp_interp(CORE, p) * g0)
            dmc_dt = -mdot_c
        else:
            Tc, dmc_dt, m_c = 0.0, 0.0, 0.0

        # second stage
        if (t >= T_IGN) and (m_s > 0):
            Ts_max = SECOND['T_vac'] - SECOND['A_e'] * p
            Ts     = second_throttle(t, throttle_second) * Ts_max
            mdot_s = Ts / (SECOND['Isp_vac'] * g0)
            dms_dt = -mdot_s
        else:
            Ts, dms_dt, m_s = 0.0, 0.0, 0.0

        # masses, forces
        mass = (BOOSTER['m_dry'] * boosters_on + m_b) + \
               (CORE   ['m_dry'] * core_on    + m_c) + \
               (SECOND ['m_dry']               + m_s) + P

        A_eff = (np.pi*d_core**2/4) \
              + ((2*np.pi*d_booster**2/4) if boosters_on else 0.)
        rho   = density_ussa(z)
        Cd = 0.4 if z < 30e3 else (0.1 if z < 70e3 else 0.0)
        D     = 0.5 * rho * Cd * A_eff * v**2
        Dx, Dz = -D * vx/v, -D * vz/v

        T  = Tb + Tc + Ts
        Tx = T * np.cos(gam)
        Tz = T * np.sin(gam)

        #  kinematics
        dxdt  = vx
        dzdt  = vz
        dvxdt = (Tx + Dx) / mass
        dvzdt = (Tz + Dz) / mass - g

        return [dxdt, dzdt, dvxdt, dvzdt,
                dmb_dt, dmc_dt, dms_dt]

    # integrate
    S0 = [0, 0, 0, 0, BOOSTER['m_prop'], CORE['m_prop'], SECOND['m_prop']]
    #t_end = T_IGN + knots_second[-1]
    t_end = SECO2
    sol   = solve_ivp(rhs, [0, t_end], S0,
                      max_step=5,  rtol=1e-3, atol=1e-4)

    _sim_cache[key] = (sol, P)
    return sol, P

#
# 3.  Objective: maximize payload
#

MONO_SCALE = 1e3
ALT_SCALE  = 1e5

def objective(theta):
    _, P = simulate(theta)
    return -P / 1000

#
# 4.  Altitude-monotonicity constraint  (dz ≥ 0 everywhere)
#
#def alt_monotone(theta):
#    sol, _ = simulate(theta)
#    return (np.min(np.diff(simulate(theta)[0].y[1]))) / MONO_SCALE         # must be ≥ 0

def alt_monotone(theta):
    sol, _ = simulate(theta)
    t = sol.t
    z = sol.y[1]
    z_powered = z[t <= SECO2]  # only during burns
    dz = np.diff(z_powered)
    return np.min(dz) / MONO_SCALE

#def final_altitude(theta):
#    sol, _ = simulate(theta)
#    return (simulate(theta)[0].y[1, -1] - ALT_TARGET) / ALT_SCALE

def final_altitude(theta):
    sol, _ = simulate(theta)
    t = sol.t
    z = sol.y[1]
    z_seco2 = np.interp(SECO2, t, z)
    return (z_seco2 - ALT_TARGET) / ALT_SCALE

cons = (NonlinearConstraint(fun=alt_monotone, lb=0.0, ub=np.inf),
       NonlinearConstraint(final_altitude,  -1, 1),
)   # dz ≥ 0

#
# 5.  Bounds: every throttle value in [0.4, 1]
#
lb = np.concatenate([0*np.ones_like(theta0[:-1]), [0.0/PAYLOAD_MAX]])
ub = np.concatenate([1.0*np.ones_like(theta0[:-1]), [1]])
bnds = Bounds(lb, ub)

i_last_boost = sl_b.stop - 1
i_last_core  = sl_c.stop - 1
lb[i_last_boost] = ub[i_last_boost] = 0.0
lb[i_last_core]  = ub[i_last_core]  = 0.0
lb[sl_s.stop-1] = ub[sl_s.stop-1]

#
# 6.  Kick off the optimiser
#
_sim_cache.clear()
result = minimize(fun=objective,
                  x0=theta0,
                  method='SLSQP',
                  bounds=bnds,
                  constraints=cons,
                  options={'eps'    : 1e-6,
                           'maxiter': 400,
                           'ftol'   : 1e-2,
                           'disp'   : True})

print("\n⇢ maximum payload:", result.x[p_idx] * PAYLOAD_MAX/1e3, "tonnes")
best_sol, _ = simulate(result.x)

def thrust_profile(sol, theta):
    t, z = sol.t, sol.y[1]
    p    = pressure_ussa(z)

    tb = np.interp(t,              knots_booster, theta[sl_b])
    tc = np.interp(t,              knots_core,    theta[sl_c])
    ts = np.array([second_throttle(ti, theta[sl_s]) for ti in t])
    #on2 = t >= T_IGN
    #ts[on2] = np.interp(t[on2] - T_IGN, knots_second, theta[sl_s])

    booster_on = (t <= BOOSTER['burn'])
    core_on    = (t <= CORE['burn'])

    Tb = booster_on * tb * thrust_interp(BOOSTER, z)
    Tc = core_on    * tc * thrust_interp(CORE,    z)
    Ts = ts * (SECOND['T_vac'] - SECOND['A_e'] * p)

    return Tb + Tc + Ts, Tb, Tc, Ts


best_sol, _   = simulate(result.x)
T_tot, Tb, Tc, Ts = thrust_profile(best_sol, result.x)

import matplotlib.pyplot as plt
plt.figure(figsize=(8,4))
plt.plot(best_sol.t, T_tot/1e6,      label='total')
plt.plot(best_sol.t, Tb/1e6,  '--',  label='boosters')
plt.plot(best_sol.t, Tc/1e6,  '--',  label='core')
plt.plot(best_sol.t, Ts/1e6,  '--',  label='2 stage')
plt.xlabel('time, s');  plt.ylabel('thrust, MN')
plt.title('Thrust schedule')
plt.legend();  plt.tight_layout();  plt.show()



print("\nOptimization exit-status:", result.message)
print("Maximum Pyload Mass  :", -result.fun, "kg")

# Altitude
plt.figure()
plt.plot(best_sol.t, best_sol.y[1]/1e3)
h_seco2 = np.interp(SECO2, best_sol.t, best_sol.y[1])        # m
plt.plot(SECO2, h_seco2/1e3, 'ro', zorder=6)                 # red dot
plt.text(SECO2, h_seco2/1e3 + 30, f'{h_seco2/1e3:.0f} km',
         ha='center', color='red')
plt.xlabel('time, s'); plt.ylabel('altitude, km')
plt.title('Altitude Profile (SLSQP)')
plt.axvline(BOOSTER['burn'], ls='--', color='k', label='booster sep')
plt.axvline(CORE['burn'], ls=':', color='tab:purple', label='core sep')
plt.axvline(T_IGN, ls='--', color='r', label='2nd Stage 1st Ign.')
plt.axvline(SECO1, ls=':', color='g', label='SECO1')
plt.axvline(T_IGN_2, ls=':', color='b', label='2nd Stage 2nd Ign')
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()